In [1]:
import pandas as pd
import numpy as np

In [151]:
def my_process(magazine):
    all_pen_year = []
    all_apple_year = []
    all_apple_year_st = []
    all_pen_year_st = []
    
    for j in magazine:
        d = []
        year = []
#         читаем и переименовываем inventory
        inventory = pd.read_csv('artem-pyanykh-cmc-prac-task3-seed17/out/input/MS-{0}-inventory.csv'.format(j))
        inventory.index = inventory['date']
        inventory = inventory.drop(inventory.columns[[0]], axis='columns')
        inventory.rename(columns={'pen': 'pen_inventory', 'apple': 'apple_inventory'}, inplace=True)
#         читаем и sell и создаем таблицы pen и apple продаж по дням
        sell = pd.read_csv('artem-pyanykh-cmc-prac-task3-seed17/out/input/MS-{0}-sell.csv'.format(j))
        sell.index = sell['date']
        sell = sell.drop(sell.columns[[0]], axis='columns')

        pen1 = sell[sell['sku_num'].str[6]=='p']
        t = pd.DataFrame(pen1.index.value_counts()).sort_index()
        pen = pd.DataFrame({'pen_sell':t['date']})

        ap1 = sell[sell['sku_num'].str[6]=='a']
        t = pd.DataFrame(ap1.index.value_counts()).sort_index()
        apple = pd.DataFrame({'apple_sell':t['date']})
        
#         считаем продажи яблок и ручек в текущем магазине за каждый год
        i = 1
        cur = pen['pen_sell'][0]
        pen_year = []
        while(i<pen.index.shape[0]):
            while(i<pen.index.shape[0] and pen.index[i].split('-')[0]==pen.index[i-1].split('-')[0]):
                cur += pen['pen_sell'][i]
                i=i+1
            pen_year.append(cur)
            cur = 0 
            i=i+1
        i = 1
        cur = apple['apple_sell'][0]
        apple_year = []
        while(i<apple.index.shape[0]):
            while(i<apple.index.shape[0] and apple.index[i].split('-')[0]==apple.index[i-1].split('-')[0]):
                cur += apple['apple_sell'][i]
                i=i+1
            apple_year.append(cur)
            cur = 0 
            year.append(apple.index[i-1].split('-')[0])
            i=i+1
#         читаем и переименовываем supply
        supply= pd.read_csv('artem-pyanykh-cmc-prac-task3-seed17/out/input/MS-{0}-supply.csv'.format(j))
        supply.index = supply['date']
        supply = supply.drop(supply.columns[[0]], axis='columns')
        supply.rename(columns={'pen': 'pen_supply', 'apple': 'apple_supply'}, inplace=True)
#         соединяем таблицы вместе 
        app = pen.join(apple, how='outer')
        app1 = app.join(supply, how='outer')
        result = app1.join(inventory, how='outer')
        result = result.fillna(0)

        del inventory
        del sell
        del supply
        del t
        del pen1
        del ap1
#         считаем apple по дням
        apple_d = []
        l1_a = np.array(result['apple_inventory'])
        l2_a = np.array(result['apple_sell'])
        l3_a = np.array(result['apple_supply'])
        inv = 0
        sup = 0
        if l1_a[0]:
            apple_d.append(l1_a[0])
        else:
            apple_d.append(l3_a[0]-l2_a[0])
        for i in range(1,len(l2_a)):
            if l1_a[i]:
                apple_d.append(l1_a[i])
            else:
                if l3_a[i]:
                    sup = l3_a[i]
                    apple_d.append(apple_d[i-1]+sup-l2_a[i])
                else:
                    apple_d.append(apple_d[i-1]-l2_a[i])
#         считаем pen по дням
        pen_d = []
        l1_p = np.array(result['pen_inventory'])
        l2_p = np.array(result['pen_sell'])
        l3_p  = np.array(result['pen_supply'])
        inv = 0
        sup = 0
        if l1_p[0]:
            pen_d.append(l1_p[0])
        else:
            pen_d.append(l3_p[0]-l2_p[0])
        for i in range(1,len(l1_p)):
            if l1_p[i]:
                pen_d.append(l1_p[i])
            else:
                if l3_p[i]:
                    sup = l3_p[i]
                    pen_d.append(pen_d[i-1]+sup-l2_p[i])
                else:
                    pen_d.append(pen_d[i-1]-l2_p[i])
                    
#           считаем украденный товар     
        i = 0
        cur_inv_p = 0
        cur_inv_a = 0
        steal_pen = []
        steal_ap = []
        cur_sell_p = 0
        cur_sup_p = 0
        cur_sell_ap = 0
        cur_sup_ap = 0
        mounth_ind = []
        drop = []
        d = 0
        while i<len(l2_p):
            while(l1_p[i] < 1):
                cur_sell_p +=l2_p[i]
                cur_sup_p +=l3_p[i]
                cur_sell_ap +=l2_a[i]
                cur_sup_ap +=l3_a[i]
                i +=1
            mounth_ind.append(result.index[i])
            steal_pen.append(-l1_p[i] + cur_inv_p - cur_sell_p - l2_p[i] + cur_sup_p )
            steal_ap.append(-l1_a[i] + cur_inv_a - cur_sell_ap - l2_a[i] + cur_sup_ap )
            if not(-l1_p[i] + cur_inv_p - cur_sell_p - l2_p[i] + cur_sup_p 
                 -l1_a[i] + cur_inv_a - cur_sell_ap - l2_a[i] + cur_sup_ap):
                drop.append(d)
            d +=1
            cur_inv_p = l1_p[i]
            cur_inv_a = l1_a[i]
            i +=1
            cur_sell_p = 0
            cur_sup_p = 0
            cur_sell_ap = 0
            cur_sup_ap = 0
        st_year_a = []
        st_year_p = []
        for i in range(10):
            st_m_p = 0
            st_m_a = 0
            for t in range(12):
                st_m_p += steal_pen[i*10+t]
                st_m_a += steal_ap[i*10+t]
            st_year_a.append(st_m_a)
            st_year_p.append(st_m_p)

#         преобразовываем в csv файл
        daily = pd.DataFrame({'date':result.index,'apple':apple_d,'pen':pen_d})
        daily.to_csv('artem-pyanykh-cmc-prac-task3-seed17/out/answer2/MS-{0}-daily.csv'.format(j))
        steal = pd.DataFrame({'date':mounth_ind,'apple':steal_ap,'pen':steal_pen})
        if drop:
            steal.drop(drop,inplace=True)
        steal.to_csv('artem-pyanykh-cmc-prac-task3-seed17/out/answer2/MS-{0}-steal.csv'.format(j))
        del result
#         добавляем в списки
        all_apple_year.append(apple_year)
        all_pen_year.append(pen_year)
        all_apple_year_st.append(st_year_a)
        all_pen_year_st.append(st_year_p)
    return year,all_apple_year,all_pen_year,all_apple_year_st,all_pen_year_st

In [152]:
m = ["b1","b2","m1","m2","s1","s2","s3","s4","s5"]

In [153]:
# year - список лет. pen - список по магазинам проданных ручек за каждый год, apple - аналогично
year,pen,apple,pen_s,apple_s = my_process(m)

In [149]:
def check(magazine):
    for j in magazine:
#         сверяем с ответами
        answer = pd.read_csv('artem-pyanykh-cmc-prac-task3-seed17/out/answer/MS-{0}-steal.csv'.format(j))
        my_answer = pd.read_csv('artem-pyanykh-cmc-prac-task3-seed17/out/answer2/MS-{0}-steal.csv'.format(j))
        ans_pen = np.array(answer['pen'])
        my_ans_pen = np.array(my_answer['pen'])   
        ans_ap = np.array(answer['apple']) 
        my_ans_ap = np.array(my_answer['apple'])   
        for i in range(len(ans_pen)-1):
            if (ans_pen[i]!=my_ans_pen[i] or ans_ap[i]!=my_ans_ap[i]):
                print("ERROR!!!",i,j)
                break

In [150]:
check(m)

In [154]:
#считаем сумму проданных яблок по всем магазинам за каждый год
cur = 0
cur_s = 0
ap = []
ap_s = []
for j in range(len(year)):
    for i in apple:
        cur +=i[j]
    ap.append(cur)
    for i in apple_s:
        cur_s +=i[j]
    ap_s.append(cur_s)
cur = 0
pe = []
cur_s = 0
pe_s = []
for j in range(len(year)):
    for i in pen:
        cur +=i[j]
    pe.append(cur)
    for i in pen_s:
        cur_s +=i[j]
    pe_s.append(cur_s)

In [155]:
result = pd.DataFrame({'year':year,'apple_sold':ap,'pen_sold':pe,'apple_steal':ap_s,'pen_steal':pe_s})

In [156]:
result

,year,apple_sold,pen_sold,apple_steal,pen_steal
0,2006,155633,2152006,461.0,418.0
1,2007,309930,4296604,833.0,811.0
2,2008,464127,6454522,1203.0,1187.0
3,2009,619096,8601264,1619.0,1595.0
4,2010,774192,10745191,2044.0,2019.0
5,2011,927927,12894116,2497.0,2463.0
6,2012,1083279,15048235,2949.0,2899.0
7,2013,1237354,17200181,3365.0,3268.0
8,2014,1391585,19347638,3827.0,3631.0
9,2015,1544722,21494008,4254.0,4008.0
